[lightgbmのinstall](https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html#install-using-homebrew)

LightGBMを使うには"cmake"と"libomp"のインストールが必要

In [91]:
import lightgbm as lgb
import pandas as pd
import re
import time

In [92]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

pd.options.display.width = 500
pd.options.display.column_space = 200

In [93]:
df = pd.read_pickle('df.pickle')
race_id_list = df.index.unique()
race_id_list

Index(['202001010101', '202001010102', '202001010103', '202001010104', '202001010105', '202001010106', '202001010107', '202001010108', '202001010109', '202001010110',
       ...
       '202002020603', '202002020604', '202002020605', '202002020606', '202002020607', '202002020608', '202002020609', '202002020610', '202002020611', '202002020612'], dtype='object', length=312)

In [94]:

race_infos = {}
# for race_id in tqdm(race_id_list):
race_id = "202110040101"
url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
html = requests.get(url)
html.encoding= "EUC_JP"
soup = BeautifulSoup(html.text, "html.parser")

test = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/horse/")})
horse_id_list = []
for a in test:
  horse_id = re.findall(r'\d+',a["href"])
  horse_id_list.append(horse_id[0])

In [95]:
horse_id_list

['2014101563',
 '2016102056',
 '2016101717',
 '2017104886',
 '2014100576',
 '2015109076',
 '2018104157',
 '2015100852',
 '2014103896',
 '2015110100',
 '2015104144',
 '2015106157']

In [96]:
race_id_lists = ['202001010101', '202001010102']

In [97]:
def scrape_race_results(race_id_lists,par_race_results={}):
  race_results = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in race_result.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      df = pd.read_html(url)[0]
      html = requests.get(url)
      html.encoding= "EUC_JP"
      soup = BeautifulSoup(html.text, "html.parser")
      # horse_id
      horse_id_list = []
      horse_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/horse/")})
      for a in horse_a_list:
        horse_id = re.findall(r'\d+',a["href"])
        horse_id_list.append(horse_id[0])
      # jockey_id
      jockey_id_list = []
      jockey_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/jockey/")})
      for u in jockey_a_list:
        # print(u)
        jockey_id = re.findall(r'\d+',u["href"])
        # print(jockey_id)
        jockey_id_list.append(jockey_id[0])
        # print(jockey_id_list)
      
      df["horse_id"] = horse_id_list
      df["jockey_id"] = jockey_id_list
      
      race_results[race_id] = df
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return race_results

In [98]:
test = scrape_race_results(race_id_lists)

100%|██████████| 2/2 [00:02<00:00,  1.41s/it]


In [99]:
test

{'202110040101':     着順  枠  馬番         馬名  性齢    斤量    騎手     タイム     着差  人気  単勝オッズ   後3F      コーナー通過順     厩舎  馬体重(増減)    horse_id jockey_id
 0    1  3   4    バトルピアサー  牝3  54.0    川島  1:46.2    NaN   1    2.7  39.0      1-1-1-1   栗東荒川  406(-2)  2017102915     01062
 1    2  8  14      バーミーズ  牝3  54.0    丹内  1:46.9      4   3    6.3  39.6      3-2-2-2  栗東飯田雄   456(0)  2017104160     01091
 2    3  8  13    ノーブルローズ  牝3  54.0    大野  1:47.0    1/2   8   19.1  38.8   14-14-11-5  美浦高柳瑞  446(-2)  2017103953     01096
 3    4  4   6  グッドラックスター  牝3  54.0   荻野琢  1:47.1    1/2   2    3.9  39.1      5-5-8-8   栗東野中  434(-6)  2017106389     01112
 4    5  3   3     デルマルビー  牝3  54.0   横山和  1:47.3  1.1/4   6   16.8  39.2    8-11-10-3   栗東千田  462(-4)  2017100813     01140
 5    6  7  12    アマデトワール  牝3  54.0   藤岡康  1:47.8      3   5   15.9  39.6    8-8-11-10  栗東高橋康  446(+4)  2017104334     01116
 6    7  6   9  ミツルザビクトリー  牝3  52.0   △亀田  1:48.0  1.1/2  12   56.1  39.5  12-12-14-10   栗東南井  438(-4)  20171